# Access quality summary statistics approach

In [ ]:
# Local path, change this.
yellow = 'C:/Users/caspe/Desktop/yellow/'
yellow_lib = yellow + "lib/"

import sys; sys.path.append(yellow_lib) 
import sqlite3
import pandas as pd
import numpy as np
import ml_utils
from os import path
from glob import glob
import re
import sys
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error, mean_squared_error
from math import sqrt
from sqlalchemy import create_engine

In [ ]:
def get_tablename(db_connection):
    query = "SELECT name FROM sqlite_master WHERE type IN ('table') AND name NOT LIKE 'sqlite_%' ORDER BY 1;"
    table_name = pd.read_sql_query(query, db_connection)
    return table_name.values[0][0]

def clean_cols(df):
    replace_map = {}
    for col in df.columns:
        replace_map[col] = re.sub("\ |\,|\)|\(|\(|\'", "", col)
    return df.rename(columns=replace_map)

In [ ]:
folder = "C:/Users/caspe/Desktop/Paper_2_StructuralVolume/results_people/"
munis = [665, 740, 751]
target = "people"

In [ ]:
dfs = []

for muni in munis:
    results = glob(folder + f'*{str(muni)}.sqlite')

    base_file = results[0]

    base_db_cnx = sqlite3.connect(base_file)
    table_name = get_tablename(base_db_cnx)
    base_df = pd.read_sql_query(f"SELECT * FROM '{table_name}';", base_db_cnx)
    base_df = clean_cols(base_df)
    base_df = base_df[["index", "fid", "people", "area", "volume", "muni_code", "muni_name"]]

    for result in results:
        filename = os.path.splitext(os.path.basename(result))[0]
        db_cnx = sqlite3.connect(result)
        table_name = get_tablename(db_cnx)
        df = pd.read_sql_query(f"SELECT * FROM '{table_name}';", db_cnx)
        df = clean_cols(df)

        layers = "_".join(filename.split("_")[1:-1])
        
        base_df[f"{target}_{layers}"] = df[df.columns[-1]]

    dfs.append(base_df)

In [ ]:
cols = dfs[0].columns
vals = np.concatenate(dfs)
merged = pd.DataFrame(vals, columns=cols)

In [ ]:
save_folder = "C:/Users/caspe/Desktop/Paper_2_StructuralVolume/"

engine = create_engine(f"sqlite:///{save_folder}results_ss_people.sqlite", echo=True)
sqlite_connection = engine.connect()

merged.to_sql("results_ss_people", sqlite_connection, if_exists='fail')
sqlite_connection.close()

In [ ]:
volume = save_folder + "results_ss_volume.sqlite"
area = save_folder + "results_ss_area.sqlite"
people = save_folder + "results_ss_people.sqlite"

vol_cnx = sqlite3.connect(volume)
vol_table_name = get_tablename(vol_cnx)
vol_db = pd.read_sql_query(f"SELECT * FROM '{vol_table_name}';", vol_cnx)

area_cnx = sqlite3.connect(area)
area_table_name = get_tablename(area_cnx)
area_db = pd.read_sql_query(f"SELECT * FROM '{area_table_name}';", area_cnx)

people_cnx = sqlite3.connect(people)
people_table_name = get_tablename(people_cnx)
people_db = pd.read_sql_query(f"SELECT * FROM '{people_table_name}';", people_cnx)

In [ ]:
join_targets = pd.concat([vol_db, area_db[area_db.columns[8:]], people_db[people_db.columns[8:]]], axis=1)
join_targets.columns

In [ ]:
join_targets = pd.concat([vol_db, area_db[area_db.columns[8:]], people_db[people_db.columns[8:]]], axis=1)

engine = create_engine(f"sqlite:///{save_folder}results_ss4.sqlite", echo=True)
sqlite_connection = engine.connect()

join_targets.to_sql("results_ss", sqlite_connection, if_exists='fail', index=False)
sqlite_connection.close()